In [28]:
import sqlite3
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

Récupération des données

In [29]:
conn = sqlite3.connect('../../db.sqlite3')

query_pizzas = """
SELECT pizza_id, name, size, ingredients
FROM menu;
"""
pizzas_df = pd.read_sql_query(query_pizzas, conn)

query_orders = """
SELECT client_id, pizza_id, order_date
FROM orders;
"""
orders_df = pd.read_sql_query(query_orders, conn)


Application de TF-IDF

In [30]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(pizzas_df['ingredients'])

tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_feature_names, index=pizzas_df['pizza_id'])

Exemple recommandation

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcul de la matrice de similarité
similarity_matrix = cosine_similarity(tfidf_matrix)

pizza_id_to_recommend = "green_garden_l"  # ID de la pizza pour laquelle on fait des recommandations
pizza_index = pizzas_df[pizzas_df['pizza_id'] == pizza_id_to_recommend].index[0]

# Calcul des similarités et tri par ordre décroissant
similar_pizzas = list(enumerate(similarity_matrix[pizza_index]))
similar_pizzas = sorted(similar_pizzas, key=lambda x: x[1], reverse=True)

recommended_pizzas = []
seen_names = set()

print(f"Les pizzas similaires à '{pizzas_df.loc[pizza_index, 'name']}' :")
for idx, score in similar_pizzas:
    pizza_name = pizzas_df.loc[idx, 'name']
    if pizza_name not in seen_names:  
        recommended_pizzas.append((pizza_name, score))
        seen_names.add(pizza_name)
    if len(recommended_pizzas) == 5:  
        break

for pizza_name, score in recommended_pizzas:
    print(f"- {pizza_name} (score de similarité : {score:.2f})")


Les pizzas similaires à 'The Green Garden Pizza' :
- The Green Garden Pizza (score de similarité : 1.00)
- The Spinach and Feta Pizza (score de similarité : 0.68)
- The Mediterranean Pizza (score de similarité : 0.44)
- The Pepperoni, Mushroom, and Peppers Pizza (score de similarité : 0.42)
- The Vegetables + Vegetables Pizza (score de similarité : 0.42)


In [32]:
conn.close()